In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cufft-

In [2]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

In [3]:
tokenizer = AutoTokenizer.from_pretrained("stevhliu/my_awesome_model")
model = AutoModelForSequenceClassification.from_pretrained("stevhliu/my_awesome_model")
model.config.id2label = {0: 'Negative', 1: 'Positive'}
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

tokenizer_config.json:   0%|          | 0.00/360 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

2025-05-15 14:52:01.747022: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747320721.934255      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747320721.987082      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [4]:
sample = "This movie was surprisingly good and had great acting!"

sample_inputs = tokenizer(sample, return_tensors="pt", truncation=True, max_length=128).to(device)
sample_input_ids = sample_inputs["input_ids"]
sample_attention_mask = sample_inputs["attention_mask"]

sample_embeddings = model.get_input_embeddings()(sample_input_ids)
sample_embeddings.retain_grad()

In [5]:
outputs = model(inputs_embeds=sample_embeddings, attention_mask=sample_attention_mask, labels=torch.tensor([1]).to(device))
loss = outputs.loss
loss.backward()

In [6]:
epsilon = 0.1
grad = sample_embeddings.grad.detach()
sample_perturbed_embeddings = sample_embeddings + epsilon * grad.sign()

In [7]:
embedding_matrix = model.get_input_embeddings().weight.detach()
normalized_matrix = F.normalize(embedding_matrix, p=2, dim=1)

def nearest_token(embedding):
    embedding = F.normalize(embedding, p=2, dim=-1)
    similarities = torch.matmul(normalized_matrix, embedding.T)
    top_ids = similarities.argmax(dim=0)
    return top_ids

nearest_ids = nearest_token(sample_perturbed_embeddings.squeeze(0))
perturbed_tokens = tokenizer.convert_ids_to_tokens(nearest_ids)
original_tokens = tokenizer.convert_ids_to_tokens(sample_input_ids.squeeze(0))

In [8]:
def prediction(embedding):
    with torch.no_grad():
        output = model(inputs_embeds=embedding, attention_mask=sample_attention_mask)
        pred = torch.argmax(output.logits, dim=1).item()
        return model.config.id2label[pred]

In [9]:
print("Original Embeddings")
print(sample_embeddings)
print()

print("Perturbed Embeddings")
print(sample_perturbed_embeddings)
print()

print("Loss gradient")
print(grad)
print()

print("Loss gradient sign")
print(grad.sign())
print()

print("Original vs FGSM Perturbed Tokens")
for orig, pert in zip(original_tokens, perturbed_tokens):
    print(f"{orig:15} -> {pert}")
print()

print("Pre-Training")
print("Original prediction:")
original_pred = prediction(sample_embeddings)
print(original_pred)
print()

print("Adversarial prediction:")
adversarial_pred = prediction(sample_perturbed_embeddings)
print(adversarial_pred)
print()

Original Embeddings
tensor([[[ 0.0407, -0.0142, -0.0190,  ...,  0.0621,  0.0214,  0.0239],
         [-0.0541,  0.0139,  0.0038,  ..., -0.0125, -0.0276,  0.0150],
         [-0.0125, -0.0161, -0.0188,  ..., -0.0295,  0.0066, -0.0579],
         ...,
         [-0.0245, -0.0397, -0.0249,  ..., -0.0645,  0.0278, -0.0150],
         [ 0.0294, -0.0239, -0.0469,  ..., -0.0121,  0.0044,  0.0039],
         [-0.0181, -0.0103, -0.0098,  ..., -0.0235,  0.0053, -0.0081]]],
       device='cuda:0', grad_fn=<EmbeddingBackward0>)

Perturbed Embeddings
tensor([[[-0.0593,  0.0858, -0.1190,  ..., -0.0379,  0.1214, -0.0761],
         [-0.1541,  0.1139, -0.0962,  ..., -0.1125,  0.0724, -0.0850],
         [-0.1125,  0.0839, -0.1188,  ...,  0.0705,  0.1066, -0.1579],
         ...,
         [ 0.0755, -0.1397, -0.1249,  ...,  0.0355, -0.0722, -0.1150],
         [ 0.1294,  0.0761, -0.1469,  ...,  0.0879,  0.1044, -0.0961],
         [-0.1181,  0.0897, -0.1098,  ...,  0.0765,  0.1053,  0.0919]]],
       device='cuda:

In [10]:
from datasets import Dataset
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [11]:
dataset = load_dataset("imdb", split="train")

df = dataset.to_pandas()
df, df_test = train_test_split(df, test_size=0.9, stratify=df['label'], random_state=42)
print("Label distribution:", df["label"].value_counts())

df_test, _ = train_test_split(df_test, test_size=0.95, stratify=df_test['label'], random_state=42)

dataset = Dataset.from_pandas(df)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256).to(device)

dataset = dataset.map(tokenize, batched=True)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Label distribution: label
0    1250
1    1250
Name: count, dtype: int64


Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [12]:
def fgsm_attack(embeddings, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_embeddings = embeddings + epsilon * sign_data_grad
    return perturbed_embeddings

In [13]:
def evaluate(texts, labels):
    model.eval()

    correct = 0
    adv_correct = 0

    for text, label in tqdm(zip(texts, labels), total=len(texts)):
        inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=256).to(device)
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]
        label_tensor = torch.tensor([label]).to(device)

        embeddings = model.get_input_embeddings()(input_ids)
        embeddings.retain_grad()

        clean_pred = prediction(embeddings)
        if clean_pred == model.config.id2label[label]:
            correct += 1

        outputs = model(inputs_embeds=embeddings, attention_mask=attention_mask, labels=label_tensor)
        loss = outputs.loss
        loss.backward()

        data_grad = embeddings.grad.detach()
        perturbed_embeddings = fgsm_attack(embeddings, 0.1, data_grad)

        adv_pred = prediction(perturbed_embeddings)
        if adv_pred == model.config.id2label[label]:
            adv_correct += 1

    clean_acc = correct / len(labels)
    adv_acc = adv_correct / len(labels)
    
    return clean_acc, adv_acc

In [14]:
pre_clean_acc, pre_adv_acc = evaluate(df_test["text"], df_test["label"])

100%|██████████| 1125/1125 [00:39<00:00, 28.29it/s]


In [15]:
optimizer = AdamW(model.parameters(), lr=2e-5)
model.train()

epsilon = 0.1
alpha = 0.5

In [16]:
for epoch in range(1):
    loop = tqdm(dataloader, desc=f"Epoch {epoch + 1}")

    correct_clean = 0
    correct_adv = 0
    total = 0

    for batch in loop:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()

        embeddings = model.get_input_embeddings()(input_ids)
        embeddings.requires_grad_()
        embeddings.retain_grad()

        outputs_clean = model(inputs_embeds=embeddings, attention_mask=attention_mask, labels=labels)
        loss_clean = outputs_clean.loss

        loss_clean.backward(retain_graph=True)
        data_grad = embeddings.grad.detach()

        perturbed_embeddings = fgsm_attack(embeddings, epsilon, data_grad)
        
        outputs_perturbed = model(inputs_embeds=perturbed_embeddings, attention_mask=attention_mask, labels=labels)
        loss_perturbed = outputs_perturbed.loss

        total_loss = loss_clean + alpha * loss_perturbed

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        pred_clean = outputs_clean.logits.argmax(dim=1)
        pred_adv = outputs_perturbed.logits.argmax(dim=1)
        correct_clean += (pred_clean == labels).sum().item()
        correct_adv += (pred_adv == labels).sum().item()
        total += labels.size(0)

        loop.set_postfix(loss=total_loss.item(), acc_clean=100 * correct_clean / total, acc_adv=100 * correct_adv / total)

Epoch 1: 100%|██████████| 157/157 [01:28<00:00,  1.78it/s, acc_adv=87.8, acc_clean=94.9, loss=0.0248]


In [17]:
post_clean_acc, post_adv_acc = evaluate(df_test["text"], df_test["label"])

100%|██████████| 1125/1125 [00:39<00:00, 28.38it/s]


In [18]:
print("Pre-training accuracy vs Post-training accuracy")
print(f"Clean Accuracy: \t {100 * pre_clean_acc:.2f}% \t {100 * post_clean_acc:.2f}%")
print(f"Adversarial Accuracy: \t {100 * pre_adv_acc:.2f}% \t {100 * post_adv_acc:.2f}%")

Pre-training accuracy vs Post-training accuracy
Clean Accuracy: 	 95.73% 	 91.38%
Adversarial Accuracy: 	 10.84% 	 98.40%


In [19]:
model.eval()

print("Post-Training")
print("Original prediction:")
original_pred = prediction(sample_embeddings)
print(original_pred)
print()

print("Adversarial prediction:")
adversarial_pred = prediction(sample_perturbed_embeddings)
print(adversarial_pred)
print()

Post-Training
Original prediction:
Positive

Adversarial prediction:
Positive

